In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_huggingface import HuggingFacePipeline ,HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import (
    FewShotPromptTemplate,
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate
)
import bitsandbytes as bnb
import pickle
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from langchain_community.document_transformers import LongContextReorder

from faiss_module import load_and_vectorize,load_chunks_make_docdb
from model import setup_llm_pipeline
from save import save
from seed import seed_everything

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_dict(dir='train.csv'):
    df = pd.read_csv(dir)
    df.drop('SAMPLE_ID', axis=1, inplace=True)
    
    return df.to_dict(orient='records')

def make_fewshot_prompt(fewshot_vectordb, k = 10):
    # Semantic Similarity Example Selector 설정
    example_prompt = PromptTemplate.from_template("<|start_header_id|>user<|end_header_id|>: <|begin_of_text|>{Question}<|end_of_text|>\n<|start_header_id|>assistant<|end_header_id|>: <|begin_of_text|>{Answer}<|end_of_text|>")

    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_vectordb,
        k=k,
    )

    # FewShotPromptTemplate 생성
    fewshot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        suffix="Question: {input}",
        input_variables=["input"],
    )
    return fewshot_prompt

def make_fewshot_string(fewshot_prompt, train_retriever, buff):
    ex_qa = fewshot_prompt.invoke({"input": buff['Question']}).to_string()
    fewshot_list = ex_qa.split('\n\n')[:-1]
    for i, entry in enumerate(fewshot_list):
        question = entry.split('\n')[0]
        question = question.replace('Question: ', '')
        retrieved_docs = train_retriever.invoke(question)
        num = "Example {}\n".format(i+1)
        fewshot_list[i] = num + "<|start_header_id|>context<|end_header_id|>: <|begin_of_text|>" + entry + '<|end_of_text|>\n\n##################################################'
    return str(fewshot_list)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n\n'
    return context

def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')
fewshot_prompt = make_fewshot_prompt(fewshot_db)

train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1})

test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
test_retriver = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 3})

train_dict = make_dict('train.csv')
test_dict = make_dict('test.csv')


Loading FAISS DB from: ./fewshot_faiss_db
Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


In [3]:

def run(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):

    
    llm = setup_llm_pipeline(model_id)
    # reordering = LongContextReorder()
    results =[]
    iii = [33,35,37,48,57,62,70]
    for i in (iii):
        
        fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
        # print(fewshot_str)
        
        full_template = """
##################################################
You are the financial literacy expert who helps me with my financial literacy Q&As.
You earn 10 points when you answer me and follow the rules and lose 12 points when you don't.
##################################################

""" +"""Here are some rules you should follow.

Rule 1: Be sure to utilize retrieved contexts for your answers.
Rule 2: The most important thing is to be concise and relevant in your answers. 
Rule 3: Answers must be written in Korean.
Rule 4: Use fewer than 128 words.
Rule 5: If you can't answer that, try summarizing the context and make it a 1-2 Sentence summary.

##################################################
context: 
{context}

<|start_header_id|>user<|end_header_id|>: <|begin_of_text|>{input}<|end_of_text|>

<|start_header_id|>assistant<|end_header_id|>: 
"""
        prompt = PromptTemplate.from_template(full_template)
        qa_chain = (
        {
            "context": test_retriver | format_docs,
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )
        print(f"Q{i}================================================")
        print("Questions: ",test_dict[i]['Question'])
        answer = qa_chain.invoke(test_dict[i]['Question'])
        #answer = extract_answer(answer)
        results.append({
            "Question": test_dict[i]['Question'],
            "Answer": answer,
            "Source": test_dict[i]['Source']
            })
        print("Answer: ",results[-1]['Answer'])
        #print(results[-1]['Source'])
run(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.69s/it]
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Q33================================================
Questions:  에너지바우처 사업의 향후 기대효과는 무엇인가요?
Answer:  에너지바우처 사업의 향후 기대 효과로는 재정 관리 강화가 있습니다. '국가재정법' 개정이 이루어졌으며, 동 법에 따른 재정 사업 평가와 개별 법령에 따르며 수행되는 평가 대상 중복을 최소화하기 위해 규정을 신선했습니다.
Q35================================================
Questions:  21년 국정감사에서 에너지 바우처 사업에 대한 주요 지적사항은 무엇이었나요?
Answer:  저소득층 폭염피해 최소화를 위해 동절기 에너지바우처 일부를 하절기에 사용할 수 있도록 제도개선을 해야한다는 것이었습니다.
Q37================================================
Questions:  에너지 바우처 사업의 향후 추진방향 중 '취약계층의 에너지 비용 부담 완화'를 위한 계획은 무엇이었나요?
Answer:  에너지 바우처 사업의 취약 계층 에너지 비용 부담 완화를 위해 다음과 같은 계획들이 있었던 것 같습니다.

1. **중증 및 희귀난치성 질환자 세대의 추가**: 2018년부터 중증 및 희귀난치성 질환자를 위한 에너지 바우처 서비스를 제공하기 시작했습니다.
2. **한부모 및 소년소녀 가정 세대의 추가**: 2019년부터 한부모 또는 소년소녀 가정의 아동들을 위한 에너지 바우처 서비스를 제공하기 시작했습니다.

이러한 노력들은 취약 계층에 대한 에너지 비용 부담을 줄이고 그들의 삶을 더 편안하게 만드는 것을 목표로 하고 있습니다.
Q48================================================
Questions:  핵심재정사업 성과관리제도를 안착시키기 위해 필요한 노력과 성과 정보를 학습의 도구로 활용하는 방안은 무엇인가?
Answer:  핵심재정사업 성과관리제도의 안착을 위하여 필요한 